In [1]:
import tempfile

from diffpy.structure import Atom, Lattice, Structure
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
import matplotlib.tri as tri
from scipy.fft import fft2, ifft2


# Colorisation and visualisation
from matplotlib.colors import to_rgb
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.lines import Line2D

In [2]:
def load_and_process_data(filepath):
    # Load the data from the file
    data = np.loadtxt(filepath, delimiter='\t', skiprows=1)
    #, max_rows=500000
    # Extract the x, y, z coordinates and the R values
    phase, x, y, z, R_flat = data[:, 0], data[:, 1], data[:, 2], data[:, 3], data[:, 7:16]
    # Reshape R_flat into a list of 3x3 matrices
    R_list = [row.reshape(3, 3) for row in R_flat]
    # Stack the 3x3 matrices vertically
    R = np.vstack([r[np.newaxis, :, :] for r in R_list])
    # Calculate the 2-norm of each column for each 3x3 matrix
    norms = np.linalg.norm(R, axis=1, keepdims=True)
    # Normalize each 3x3 matrix by its column norms
    R_normalized = R / norms
    print(np.shape(R_normalized))
    return phase, x, y, z, R_normalized

In [3]:
def compute_layer_step_sizes(x, z):
    """
    计算每一层的步长，并保存到文件中。
    """
    data = np.column_stack((x, z))
    unique_z = np.unique(z)
    
    step_sizes = []

    for z_value in unique_z:
        x_in_layer = x[z == z_value]  # 获取当前层的x坐标
        if len(x_in_layer) > 1:
            step_size = np.round(x_in_layer[1] - x_in_layer[0], 4)  # 计算步长并保留3位小数
            step_sizes.append((np.round(z_value, 4), step_size))

    # Save the step sizes to a text file
    with open('step_sizes.txt', 'w') as f:
        f.write('Layer\tZ\tStep_Size\n')
        for i, (z_value, step_size) in enumerate(step_sizes):
            f.write(f'{i + 1}\t{z_value}\t{step_size}\n')

In [4]:
from scipy.fft import fft2, ifft2
import numpy as np
import pandas as pd

def align_and_correct_ebsd_data(x, y, z, phase, R_normalized):
    # 将 R_normalized 展平为二维数组
    R_normalized = R_normalized.reshape(R_normalized.shape[0], -1)  # 变为形状 (3, 9) 的数组
    data = np.column_stack((x, y, z, phase, R_normalized))  # 将x, y, z, phase, R_normalized组合成一个数组
    
    def split_by_layer(data):
        """
        根据z值将数据分层，每层的数据存储在字典中。
        字典的键是z值，对应的值是该层的数据。
        """
        layers = {}
        unique_z = np.unique(data[:, 2])
        for z in unique_z:
            layers[z] = data[data[:, 2] == z]
        return layers

    def create_binary_image(layer_data):
        """
        创建一个二值图像，其中phase > 0的区域设为1，其他区域设为0。
        """
        df = pd.DataFrame(layer_data, columns=['X', 'Y', 'Z', 'Phase'] + [f'R{i+1}' for i in range(9)] + ['original_indices'])
        img = df.pivot(index='Y', columns='X', values='Phase').fillna(0).values
        img = (img > 0).astype(np.uint8)
        return img

    def calculate_shift_fft(reference_image, moving_image):
        """
        使用快速傅里叶变换（FFT）计算两幅图像之间的平移量。
        """
        f1 = fft2(reference_image)
        f2 = fft2(moving_image)
        cross_power_spectrum = (f1 * f2.conj()) / np.abs(f1 * f2.conj())
        cross_correlation = np.abs(ifft2(cross_power_spectrum))
        y_shift, x_shift = np.unravel_index(np.argmax(cross_correlation), cross_correlation.shape)
        y_shift -= reference_image.shape[0] // 2
        x_shift -= reference_image.shape[1] // 2
        return x_shift, y_shift

    def align_and_correct_coordinates(reference_image, moving_image, moving_data):
        """
        使用FFT计算的平移量对移动层的数据进行对齐。
        """
        x_shift, y_shift = calculate_shift_fft(reference_image, moving_image)
        corrected_data = moving_data.copy()
        corrected_data[:, 0] += x_shift
        corrected_data[:, 1] += y_shift
        return corrected_data

    original_indices = np.arange(data.shape[0])
    data_with_indices = np.hstack((data, original_indices.reshape(-1, 1)))
    layers = split_by_layer(data_with_indices)

    all_layer_z_values = sorted(layers.keys())
    z_values_to_use = all_layer_z_values[52:184]  # 从第53层到第184层

    reference_layer_z = z_values_to_use[0]
    reference_data = layers[reference_layer_z]
    reference_image = create_binary_image(reference_data)

    corrected_layers = {reference_layer_z: reference_data}

    for z in z_values_to_use[1:]:
        moving_data = layers[z]
        moving_image = create_binary_image(moving_data)
        corrected_data = align_and_correct_coordinates(reference_image, moving_image, moving_data)
        corrected_layers[z] = corrected_data
        reference_image = create_binary_image(corrected_data)

    corrected_data = np.vstack([corrected_layers[z] for z in sorted(corrected_layers.keys())])
    sorted_corrected_data = corrected_data[corrected_data[:, -1].argsort()]
    sorted_corrected_data = sorted_corrected_data[:, :-1]
    x_corrected = sorted_corrected_data[:, 0]
    y_corrected = sorted_corrected_data[:, 1]
    z_corrected = sorted_corrected_data[:, 2]
    phase_corrected = sorted_corrected_data[:, 3]
    R_corrected = sorted_corrected_data[:, 4:]  # 取出 R_normalized 部分

    return x_corrected, y_corrected, z_corrected, phase_corrected, R_corrected

In [5]:
def get_ipf_color(vectors):
    vectors = np.abs(vectors)
    # Convert Cartesian coordinates to spherical coordinates
    h = vectors[:,0]
    k = vectors[:,1]
    l = vectors[:,2]
    r = np.ones(len(vectors))
    theta = np.arcsin(np.dot(vectors, [1,0,0]))
    phi = np.arctan(l/k)
    phi = 0.5 * phi
    k_p = r * np.sin(theta) * np.cos(phi)
    l_p = r * np.sin(theta) * np.sin(phi)
    h_p = np.sqrt(r**2 - k_p**2 - l_p**2)
    gamma = np.arctan(h_p/k_p)
    gamma = 0.5 * gamma
    k_pp = np.sqrt((1-l**2)/(1+np.tan(gamma)**2))
    h_pp = np.tan(gamma) * k_pp
    l_pp = np.sqrt(r**2 - h_pp**2 - k_pp**2)
    hkl = np.stack((h_pp, k_pp, l_pp), axis=-1)
    cube_rgb = hkl / np.max(hkl, axis=1)[:, np.newaxis]
    
    return cube_rgb

In [6]:
def plot_3d_ipf(phase, x, y, z, R):
    
    print(np.shape(R))
    
    # 输出层数
    unique_z = np.unique(z)
    num_layers = len(unique_z)
    print(f"Number of layers (unique z values): {num_layers}")
    
    titles = ["X", "Y", "Z"]
    mask = phase != 0
    x = x[mask]
    y = y[mask]
    z = z[mask]
    R= R[mask]
    
    print(np.shape(R))
    # 输出层数
    unique_z = np.unique(z)
    num_layers = len(unique_z)
    print(f"Number of layers (unique z values): {num_layers}")
    
    fig = plt.figure(figsize=(10, 30))
    axes = [fig.add_subplot(3, 1, i+1, projection='3d') for i in range(3)]
    
    print(f"x range: {x.min()} to {x.max()}")
    print(f"y range: {y.min()} to {y.max()}")
    print(f"z range: {z.min()} to {z.max()}")
    
    for i, ax in enumerate(axes):

        ax.set_title(f"IPF-{titles[i]}")
        # Filter out points where phase is 0
        vectors = R[:, :, i]
        cube_rgb = get_ipf_color(vectors)
        # Plot the data with the specified colors
        ax.scatter(x, y, z, c=cube_rgb, alpha=1, s=1)
        # Calculate the common axis limit to keep the aspect ratio
        # Ensure the axis limits are within the specified range
        #axis_limit = min(max(np.max([x, y, z]), 20), 20)
        
        # Set the x, y, and z axis limits to the same range, ensuring it does not exceed 30
        ax.set_xlim(-100, 100)
        ax.set_ylim(-300, 300)
        ax.set_zlim(0, 10)

    plt.show()

In [7]:
def main():
    # Specify the filepath to the data file
    filepath = '3d_ebsd_20240619_R_appended_data.txt'
    # Load and process the data
    phase, x, y, z, R_normalized = load_and_process_data(filepath)
    # 计算和保存每层的步长
    #compute_layer_step_sizes(x, z)
    # 矫正层间漂移，对齐数据层
    x_a, y_a, z_a, phase_a, R_corrected = align_and_correct_ebsd_data(x, y, z, phase, R_normalized)
    # 将R_corrected重新变回n*3*3的数组
    R_corrected_reshaped = R_corrected.reshape(-1, 3, 3)
    # Plot the processed data
    plot_3d_ipf(phase_a, x_a, y_a, z_a, R_corrected_reshaped)
    # 创建合并数据
    appended_data = np.column_stack((phase_a, x_a, y_a, z_a, R_corrected))
    # Save the results as a .txt file
    header = 'Phase\tx\ty\tz\t' + '\t'.join([f'R{i+1}' for i in range(9)])
    np.savetxt('3d_ebsd_20240619_drift_modified_data.txt', appended_data, delimiter='\t', header=header, fmt='%s')

: 

In [8]:
if __name__ == "__main__":
    main()

/var/folders/ww/5vsdwswx1t510sdhmkz3673r0000gn/T/ipykernel_8390/2353716698.py:14: RuntimeWarning: invalid value encountered in divide
  R_normalized = R / norms


(17987412, 3, 3)
